In [2]:
# linear algebra
import numpy as np 

# data processing
import pandas as pd 
import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split

# data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from scipy import stats

import pycountry_convert as pc
import holidays
from datetime import date

%matplotlib inline
fig=plt.figure(figsize=(18,6))

<Figure size 1296x432 with 0 Axes>

In [3]:
hotel_data_raw = pd.read_csv('./data/hotel_bookings.csv')
hotel_data_raw.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
# features to be removed: lead_time (since lead time is talking abt the period of time between reservation and checkin), 
#                         arrival_date_year (won't be applicable in the future), company, 
#                         assigned_room_type (should be assigned after arrival)
#                         reservation_status, reservation_status_date (the latest reservation status update time)
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 300


In [5]:
# Change Data Types From Number to String
hotel_data_raw['is_repeated_guest'] = hotel_data_raw['is_repeated_guest'].astype(str)
hotel_data_raw['agent'] = hotel_data_raw['agent'].astype(str)
hotel_data_raw['is_canceled'] = hotel_data_raw['is_canceled'].astype(str)
hotel_data_raw['arrival_date_month'] = hotel_data_raw['arrival_date_month'].astype(str)
hotel_data_raw['arrival_date_day_of_month'] = hotel_data_raw['arrival_date_day_of_month'].astype(str)
hotel_data_raw['arrival_date_week_number'] = hotel_data_raw['arrival_date_week_number'].astype(str)

In [6]:
# normalization (using minmax)
# • Lead_time
# • Days_in_waiting_list
# • Previous_cancellations
# • Previous_bookings_not_canceled
# • Adr
# • Required_car_parking_spaces
# • Adults
# • Children
# • Babies
# • Stays_in_weekend_nights
# • Stays_in_week_nights
# • Required_car_parking_spaces
# • Booking_changes

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['lead_time']])
hotel_data_raw['lead_time'] = minmax_scaler.transform(hotel_data_raw[['lead_time']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['previous_cancellations']])
hotel_data_raw['previous_cancellations'] = minmax_scaler.transform(hotel_data_raw[['previous_cancellations']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['days_in_waiting_list']])
hotel_data_raw['days_in_waiting_list'] = minmax_scaler.transform(hotel_data_raw[['days_in_waiting_list']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['previous_bookings_not_canceled']])
hotel_data_raw['previous_bookings_not_canceled'] = minmax_scaler.transform(hotel_data_raw[['previous_bookings_not_canceled']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['adr']])
hotel_data_raw['adr'] = minmax_scaler.transform(hotel_data_raw[['adr']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['required_car_parking_spaces']])
hotel_data_raw['required_car_parking_spaces'] = minmax_scaler.transform(hotel_data_raw[['required_car_parking_spaces']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['adults']])
hotel_data_raw['adults'] = minmax_scaler.transform(hotel_data_raw[['adults']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['children']])
hotel_data_raw['children'] = minmax_scaler.transform(hotel_data_raw[['children']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['babies']])
hotel_data_raw['babies'] = minmax_scaler.transform(hotel_data_raw[['babies']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['stays_in_weekend_nights']])
hotel_data_raw['stays_in_weekend_nights'] = minmax_scaler.transform(hotel_data_raw[['stays_in_weekend_nights']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['stays_in_week_nights']])
hotel_data_raw['stays_in_week_nights'] = minmax_scaler.transform(hotel_data_raw[['stays_in_week_nights']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['required_car_parking_spaces']])
hotel_data_raw['required_car_parking_spaces'] = minmax_scaler.transform(hotel_data_raw[['required_car_parking_spaces']])

minmax_scaler = preprocessing.MinMaxScaler().fit(hotel_data_raw[['booking_changes']])
hotel_data_raw['booking_changes'] = minmax_scaler.transform(hotel_data_raw[['booking_changes']])

In [7]:
# One-hot encoding:
# • hotel
# • Arrival_date_month
# • Meal
# • Market_segment
# • Distribution_channel
# • Customer_type
# • Deposit_type
# • country

######
# • Week_of_day
# • Holiday_of_day
# • agent
# 	◦ Binary feature
# • Company
# 	◦ Binary feature

In [8]:
# Handle missing values
# test['Age'].fillna(test['Age'].mean(),inplace = True)
hotel_data_raw['children'].fillna(hotel_data_raw['children'].value_counts().index[0],inplace = True)
hotel_data_raw['country'].fillna(hotel_data_raw['country'].value_counts().index[0],inplace = True)
# do nothing for agent and company as its missing value take meanings

In [9]:
# Add new features
hotel_data_raw['is_company'] = (hotel_data_raw['company'].notnull())
hotel_data_raw['is_agent'] = (hotel_data_raw['agent'].notnull())

hotel_data_raw['is_diff_room_type'] = (hotel_data_raw['reserved_room_type'] != hotel_data_raw['assigned_room_type'])
hotel_data_raw_dates = hotel_data_raw['arrival_date_year'].astype(str) + ' ' + hotel_data_raw['arrival_date_month'].astype(str) + ' ' + hotel_data_raw['arrival_date_day_of_month'].astype(str)
# Portugal plus extended days most people have off
holidays_set = holidays.CountryHoliday('PTE')
hotel_data_raw['arrival_date_day_of_the_week'] = [datetime.datetime.strptime(date, '%Y %B %d').strftime('%A') for date in hotel_data_raw_dates]
hotel_data_raw['is_holiday'] = [datetime.datetime.strptime(date, '%Y %B %d').strftime('%Y-%m-%d') in holidays_set for date in hotel_data_raw_dates]


In [10]:
hotel_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 37 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  object 
 2   lead_time                       119390 non-null  float64
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  object 
 6   arrival_date_day_of_month       119390 non-null  object 
 7   stays_in_weekend_nights         119390 non-null  float64
 8   stays_in_week_nights            119390 non-null  float64
 9   adults                          119390 non-null  float64
 10  children                        119390 non-null  float64
 11  babies                          119390 non-null  float64
 12  meal            

In [11]:
hotel_data_raw.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,is_company,is_agent,is_diff_room_type,arrival_date_day_of_the_week,is_holiday
0,Resort Hotel,0,0.464043,2015,July,27,1,0.0,0.00,0.036364,0.0,0.0,BB,PRT,Direct,Direct,0,0.0,0.0,C,C,0.142857,No Deposit,nan,NaN,0.0,Transient,0.001180,0.0,0,Check-Out,2015-07-01,False,True,False,Wednesday,False
1,Resort Hotel,0,1.000000,2015,July,27,1,0.0,0.00,0.036364,0.0,0.0,BB,PRT,Direct,Direct,0,0.0,0.0,C,C,0.190476,No Deposit,nan,NaN,0.0,Transient,0.001180,0.0,0,Check-Out,2015-07-01,False,True,False,Wednesday,False
2,Resort Hotel,0,0.009498,2015,July,27,1,0.0,0.02,0.018182,0.0,0.0,BB,GBR,Direct,Direct,0,0.0,0.0,A,C,0.000000,No Deposit,nan,NaN,0.0,Transient,0.015053,0.0,0,Check-Out,2015-07-02,False,True,True,Wednesday,False
3,Resort Hotel,0,0.017639,2015,July,27,1,0.0,0.02,0.018182,0.0,0.0,BB,GBR,Corporate,Corporate,0,0.0,0.0,A,A,0.000000,No Deposit,304.0,NaN,0.0,Transient,0.015053,0.0,0,Check-Out,2015-07-02,False,True,False,Wednesday,False
4,Resort Hotel,0,0.018996,2015,July,27,1,0.0,0.04,0.036364,0.0,0.0,BB,GBR,Online TA,TA/TO,0,0.0,0.0,A,A,0.000000,No Deposit,240.0,NaN,0.0,Transient,0.019307,0.0,1,Check-Out,2015-07-03,False,True,False,Wednesday,False


In [12]:
le = preprocessing.LabelEncoder()
hotel_data_raw['hotel_encoded'] = le.fit_transform(hotel_data_raw['hotel'])
hotel_data_raw['arrival_date_month_encoded'] = le.fit_transform(hotel_data_raw['arrival_date_month'])
hotel_data_raw['meal_encoded'] = le.fit_transform(hotel_data_raw['meal'])
hotel_data_raw['market_segment_encoded'] = le.fit_transform(hotel_data_raw['market_segment'])
hotel_data_raw['distribution_channel_encoded'] = le.fit_transform(hotel_data_raw['distribution_channel'])
hotel_data_raw['customer_type_encoded'] = le.fit_transform(hotel_data_raw['customer_type'])
hotel_data_raw['deposit_type_encoded'] = le.fit_transform(hotel_data_raw['deposit_type'])
hotel_data_raw['arrival_date_day_of_the_week_encoded'] = le.fit_transform(hotel_data_raw['arrival_date_day_of_the_week'])




hotel_data_raw = pd.get_dummies(hotel_data_raw, columns=[ \
                                                         'hotel', \
                                                         'arrival_date_month', \
                                                         'meal', \
                                                         'market_segment', \
                                                         'distribution_channel', \
                                                         'customer_type', \
                                                         'deposit_type', \
                                                         'arrival_date_day_of_the_week'])

In [13]:
hotel_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 83 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   is_canceled                             119390 non-null  object 
 1   lead_time                               119390 non-null  float64
 2   arrival_date_year                       119390 non-null  int64  
 3   arrival_date_week_number                119390 non-null  object 
 4   arrival_date_day_of_month               119390 non-null  object 
 5   stays_in_weekend_nights                 119390 non-null  float64
 6   stays_in_week_nights                    119390 non-null  float64
 7   adults                                  119390 non-null  float64
 8   children                                119390 non-null  float64
 9   babies                                  119390 non-null  float64
 10  country                                 1193

In [14]:
hotel_data_raw.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,is_company,is_agent,is_diff_room_type,is_holiday,hotel_encoded,arrival_date_month_encoded,meal_encoded,market_segment_encoded,distribution_channel_encoded,customer_type_encoded,deposit_type_encoded,arrival_date_day_of_the_week_encoded,hotel_City Hotel,hotel_Resort Hotel,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,arrival_date_month_March,arrival_date_month_May,arrival_date_month_November,arrival_date_month_October,arrival_date_month_September,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,arrival_date_day_of_the_week_Friday,arrival_date_day_of_the_week_Monday,arrival_date_day_of_the_week_Saturday,arrival_date_day_of_the_week_Sunday,arrival_date_day_of_the_week_Thursday,arrival_date_day_of_the_week_Tuesday,arrival_date_day_of_the_week_Wednesday
0,0,0.464043,2015,27,1,0.0,0.00,0.036364,0.0,0.0,PRT,0,0.0,0.0,C,C,0.142857,nan,NaN,0.0,0.001180,0.0,0,Check-Out,2015-07-01,False,True,False,False,1,5,0,3,1,2,0,6,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
1,0,1.000000,2015,27,1,0.0,0.00,0.036364,0.0,0.0,PRT,0,0.0,0.0,C,C,0.190476,nan,NaN,0.0,0.001180,0.0,0,Check-Out,2015-07-01,False,True,False,False,1,5,0,3,1,2,0,6,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
2,0,0.009498,2015,27,1,0.0,0.02,0.018182,0.0,0.0,GBR,0,0.0,0.0,A,C,0.000000,nan,NaN,0.0,0.015053,0.0,0,Check-Out,2015-07-02,False,True,True,False,1,5,0,3,1,2,0,6,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
3,0,0.017639,2015,27,1,0.0,0.02,0.018182,0.0,0.0,GBR,0,0.0,0.0,A,A,0.000000,304.0,NaN,0.0,0.015053,0.0,0,Check-Out,2015-07-02,False,True,False,False,1,5,0,2,0,2,0,6,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1
4,0,0.018996,2015,27,1,0.0,0.04,0.036364,0.0,0.0,GBR,0,0.0,0.0,A,A,0.000000,240.0,NaN,0.0,0.019307,0.0,1,Check-Out,2015-07-03,False,True,False,False,1,5,0,6,3,2,0,6,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1


In [15]:
hotel_data_raw.corr()

,lead_time,arrival_date_year,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,is_company,is_agent,is_diff_room_type,is_holiday,hotel_encoded,arrival_date_month_encoded,meal_encoded,market_segment_encoded,distribution_channel_encoded,customer_type_encoded,deposit_type_encoded,arrival_date_day_of_the_week_encoded,hotel_City Hotel,hotel_Resort Hotel,arrival_date_month_April,arrival_date_month_August,arrival_date_month_December,arrival_date_month_February,arrival_date_month_January,arrival_date_month_July,arrival_date_month_June,arrival_date_month_March,arrival_date_month_May,arrival_date_month_November,arrival_date_month_October,arrival_date_month_September,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,arrival_date_day_of_the_week_Friday,arrival_date_day_of_the_week_Monday,arrival_date_day_of_the_week_Saturday,arrival_date_day_of_the_week_Sunday,arrival_date_day_of_the_week_Thursday,arrival_date_day_of_the_week_Tuesday,arrival_date_day_of_the_week_Wednesday
lead_time,1.000000,0.040142,0.085671,0.165799,0.119519,-0.037613,-0.020915,0.086042,-0.073548,0.000149,0.151464,0.170084,-0.063077,-0.116451,-0.095712,-0.126280,NaN,-0.142220,0.000905,-0.075381,0.086228,0.000349,0.013797,0.220414,0.073403,0.375667,-0.007171,0.075381,-0.075381,-0.027514,0.058121,-0.069918,-0.141459,-0.125571,0.104140,0.072029,-0.094810,0.042237,-0.063054,0.057043,0.094956,-0.038259,0.009714,0.136633,-0.097617,-0.015512,-0.041554,-0.067184,-0.165075,-0.174111,0.346275,0.146307,-0.186304,-0.003926,-0.134609,-0.161594,-0.031365,0.221734,-0.004906,0.068541,-0.031924,-0.173959,0.159548,-0.380233,0.380179,0.016587,0.010878,-0.027052,0.016487,-0.059170,0.123286,-0.056259,-0.024403
arrival_date_year,0.040142,1.000000,0.021497,0.030883,0.029635,0.054636,-0.013192,-0.119822,0.029218,0.030872,0.259095,-0.056497,0.197580,-0.013684,0.108531,-0.003020,NaN,-0.097402,-0.000664,-0.035267,-0.251837,0.065840,0.107697,0.022644,-0.006149,-0.064714,0.019540,0.035267,-0.035267,0.160090,-0.041982,-0.203670,0.137434,0.150020,0.021336,0.161462,0.148271,0.177269,-0.173943,-0.272664,-0.282448,-0.032865,-0.057964,-0.057955,0.128706,0.007213,0.019390,-0.010135,-0.025186,0.013156,-0.106530,-0.123760,0.181268,-0.006691,-0.030312,-0.000604,0.016735,0.017064,-0.010580,-0.171738,-0.010982,0.227954,-0.163588,0.065736,-0.065963,-0.000760,-0.025467,0.001557,-0.024043,0.038880,0.009648,0.012784,-0.007986
stays_in_weekend_nights,0.085671,0.021497,1.000000,0.498969,0.091871,0.045794,0.018483,-0.012775,-0.042715,0.063281,0.066749,-0.054151,0.049342,-0.018554,0.072671,-0.107542,NaN,-0.076657,-0.005107,0.186596,-0.031869,0.045744,0.115350,0.093097,-0.109220,-0.111435,-0.260299,-0.186596,0.186596,0.006758,0.048250,-0.015371,-0.020077,-0.033927,0.066460,-0.002297,-0.020628,-0.019211,-0.035051,-0.010373,0.004494,-0.066769,0.017476,0.105243,-0.035044,0.022202,0.007379,-0.045670,-0.105547,-0.024573,-0.062016,0.063606,0.060176,-0.001753,-0.087007,-0.036528,-0.013576,0.084595,0.000469,0.102038,-0.007163,0.019485,-0.064920,0.113537,-0.114275,0.001761,-0.074124,0.112630,0.228218,0.377593,-0.185184,-0.216060,-0.231465
stays_in_week_nights,0.165799,0.030883,0.498969,1.000000,0.092976,0.044203,0.020191,-0.013992,-0.048743,0.096209,0.182211,-0.002020,0.065237,-0.024859,0.068192,-0.081370,NaN,-0.091036,0.009564,0.234020,-0.025746,0.036742,0.1

In [16]:
continents = []

continent_name_code = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'AQ': 'Antarctica',
    'EU': 'Europe',
    
}

for country in hotel_data_raw["country"]:
    try:
        if country == 'CN':
            country_code == 'CN'
        elif country == 'TMP':
            country_code == 'TL'
        else:
            country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")

    except:
        print(country,'is not valid')
    
    try:
        #Antarctica
        if country_code == 'AQ':
            continent = 'AQ'
        #United States Minor Outlying Islands
        elif country_code == 'UM':
            continent = 'NA'
        #French Southern and Antarctic Lands
        elif country_code == 'TF':
            continent = 'EU'
        else:
            continent = pc.country_alpha2_to_continent_code(country_code)
    except:
        print(country_code,'is not valid for continent')

        
    continents.append(continent_name_code[continent])
    

hotel_data_raw['continent'] = continents



In [17]:
hotel_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 84 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   is_canceled                             119390 non-null  object 
 1   lead_time                               119390 non-null  float64
 2   arrival_date_year                       119390 non-null  int64  
 3   arrival_date_week_number                119390 non-null  object 
 4   arrival_date_day_of_month               119390 non-null  object 
 5   stays_in_weekend_nights                 119390 non-null  float64
 6   stays_in_week_nights                    119390 non-null  float64
 7   adults                                  119390 non-null  float64
 8   children                                119390 non-null  float64
 9   babies                                  119390 non-null  float64
 10  country                                 1193

In [18]:
hotel_data_raw['continent_encoded'] = le.fit_transform(hotel_data_raw['continent'])

hotel_data_raw = pd.get_dummies(hotel_data_raw, columns=['continent'])


In [19]:
hotel_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 91 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   is_canceled                             119390 non-null  object 
 1   lead_time                               119390 non-null  float64
 2   arrival_date_year                       119390 non-null  int64  
 3   arrival_date_week_number                119390 non-null  object 
 4   arrival_date_day_of_month               119390 non-null  object 
 5   stays_in_weekend_nights                 119390 non-null  float64
 6   stays_in_week_nights                    119390 non-null  float64
 7   adults                                  119390 non-null  float64
 8   children                                119390 non-null  float64
 9   babies                                  119390 non-null  float64
 10  country                                 1193

In [20]:
hotel_data_raw.to_csv('Hotel_Bookings_Prepared.csv')

In [22]:
i=1
fig=plt.figure(figsize=(240,100))
for col in hotel_data_raw.select_dtypes([np.number]).columns:
    ax = plt.subplot(7,11,i)
    ax.set_title(col)
    i = i+1
    hotel_data_raw[col].hist()